In [1]:
from co2data.departments.department_provider import DepartmentProvider
import sys
sys.path.insert(0, "..")
from co2data.categories.importer import create_category_provider_from_directory
from pathlib import Path

category_provider = create_category_provider_from_directory(Path("../data/categories"))
department_provider = DepartmentProvider.create_from_text_file(Path("../data/departments.txt"))

In [2]:
from co2data.pdf.pre_render_page_provider import CachingPageProvider
from co2data.pdf.filesystem_cache_page_provider import FileSystemCachePageProvider
from co2data.pdf.concrete_page_provider import ConcretePageProvider
from co2data.pdf.file_system_pdf_store import FileSystemPdfStore
from co2data.bills.bill_data_provider_factory import create_bill_data_provider

pdf_store = FileSystemPdfStore()
bill_data_provider = create_bill_data_provider(pdf_store)
page_provider = FileSystemCachePageProvider(CachingPageProvider(ConcretePageProvider(pdf_store)), Path("../datadir/.pdf_cache"))
page_provider.populate_cache(pdf_store.identifiers)


INFO:root:bills dir: /home/camma/Documents/privat/co2data_export/notebook/../datadir from (../datadir)
INFO:root:autosave: /home/camma/Documents/privat/co2data_export/notebook/../datadir/.autosave
INFO:root:state file: /home/camma/Documents/privat/co2data_export/notebook/../datadir/.autosave/saved_bill_data.json
INFO:root:statefile exists
INFO:root:BillStore: file identifier _146159.pdf already present
INFO:root:BillStore: file identifier 0000_146_09060_R_60281042-20221221.pdf already present
INFO:root:BillStore: file identifier 0000_146_09060_R_60292797-20230322.pdf already present
INFO:root:BillStore: file identifier 3231244.pdf already present
INFO:root:BillStore: file identifier 3231904.pdf already present
INFO:root:BillStore: file identifier 0000_146_09060_R_60291816-20230315.pdf already present
INFO:root:BillStore: file identifier 0000_146_09060_R_60286831-20230208.pdf already present
INFO:root:BillStore: file identifier 0000_146_09060_R_60289216-20230227.pdf already present
INFO

In [5]:
from co2data.bills.bill import Position
from co2data.bills.bill_id import BillId
from co2data.bills.bill_status import BillStatus
from ipywidgets import Layout
import ipywidgets as widgets

select_rows = 20
select_todo = widgets.Select(
    options=bill_data_provider.get_by_status(BillStatus.TO_DO),
    rows=select_rows,
    description='ToDo:',
    disabled=False,
    value=None
)
select_inprogress = widgets.Select(
    options=bill_data_provider.get_by_status(BillStatus.IN_PROGRESS),
    rows=select_rows,
    description='In Progress:',
    disabled=False,
    value=None
)
select_done = widgets.Select(
    options=bill_data_provider.get_by_status(BillStatus.DONE),
    rows=select_rows,
    description='Done:',
    disabled=False
)

select_department = widgets.Dropdown(
    options=department_provider.departments,
    description="Sektionsteil"
)

select_area = widgets.Dropdown(
    options=category_provider.get_areas(),
    description="Bereich"
)

select_category = widgets.Dropdown(
    options=[category.name for category in category_provider.get_categories_for_area(select_area.value)],
    description="Kategorie"
)

select_position = widgets.Select(
    rows=15,
)

button_position_up = widgets.Button(
    description="Up"
)

button_position_down = widgets.Button(
    description="Down"
)

position_value_default = 0
textinput_position_value = widgets.Text(
    placeholder=str(position_value_default),
    description='Wert:',
    disabled=False,
)

position_amount_default = 1
textinput_position_amount = widgets.Text(
    placeholder=str(position_amount_default),
    description='Menge:',
    disabled=False,
)

position_price_default = 0
textinput_position_price = widgets.Text(
    placeholder=str(position_price_default),
    description='Preis:',
    disabled=False,
)

radiobutton_tax = widgets.Dropdown(
    options=[('0%', 0), ('7%', 7), ('19%', 19)],
    value=0,
    description='Number:',
)

unit_label = widgets.Label(
    value="test"
)

button_add_position = widgets.Button(
    description="Add"
)

button_delete_position = widgets.Button(
    description="Delete"
)

button_done = widgets.Button(
    description="Done"
)

button_save_comment = widgets.Button(
    description="Save comment"
)

comment_area = widgets.Textarea(
    value='',
    placeholder='Type a comment for the bill',
    description='String:',
    disabled=False,
    rows=10
)

button_previous_page = widgets.Button(
    description="Previous"
)

button_next_page = widgets.Button(
    description="Next"
)

current_page_label = widgets.Label(value="")

log_output = widgets.Output()

image = widgets.Image(width=700, height=1000)

class CurrentBill:

    def __init__(self) -> None:
        self.bill_id = None
        self._page_nr = 0

    @property
    def id(self) -> BillId | None:
        return self.bill_id

    def set_id(self, bill_id: BillId) -> None:
        self.bill_id = bill_id
        log(f"current bill: {self.bill_id}")
        self.page = 0
        
    @property
    def page(self) -> int:
        return self._page_nr
    
    @page.setter
    def page(self, page_nr: int) -> None:
        self._page_nr = page_nr
        current_page_label.value = f"{self._page_nr + 1}  /  {str(page_provider.get_nr_of_pages(bill_data_provider.get_file_identifier(self.bill_id)))}"
        log(f"current page {self._page_nr + 1}")
        show_pdf(self.bill_id, self._page_nr)
        

current_bill = CurrentBill()

def handle_todo_selection(event):
    if event.new is None:
        return
    handle_new_current_bill(event)
    select_done.value = None
    select_inprogress.value = None
    

def handle_inprogress_selection(event):
    if event.new is None:
        return
    handle_new_current_bill(event)
    select_done.value = None
    select_todo.value = None
    

def handle_done_selection(event):
    if event.new is None:
        return
    handle_new_current_bill(event)
    select_todo.value = None
    select_inprogress.value = None
    

def handle_new_current_bill(event):
    current_bill.set_id(event.new)
    update_positions()
    update_comment_area()
    radiobutton_tax.index = 0
    
def handle_area_selection(event):
    log(f"new area: {event.new}")
    select_category.options = [category.name for category in category_provider.get_categories_for_area(select_area.value)]
    

def handle_new_category(event):
    category = category_provider.get_by_name(event.new)
    log(f"new category: {category.name} with unit {category.unit}")
    unit_label.value = category.unit

def log(message: str) -> None:
    with log_output:
        print(message)

def show_pdf(bill_id, page):
    img = page_provider.get_page(bill_data_provider.get_file_identifier(bill_id), page)
    image.value = img.make_blob(format="png")
    image.format = "png"
    
def handle_previous_page(button: widgets.Button) -> None:
    if current_bill.page > 0:
        current_bill.page -= 1
    else:
        log("already at page 1")

def handle_next_page(button: widgets.Button) -> None:
    if current_bill.page + 1 < page_provider.get_nr_of_pages(bill_data_provider.get_file_identifier(current_bill.bill_id)):
        current_bill.page += 1
    else:
        log("already on last page")
        
def handle_add_position_press(button: widgets.Button) -> None:
    add_position()
    
def on_position_value_submit(sender) -> None:
    add_position()
    
def on_position_amount_submit(sender) -> None:
    add_position()
    
def on_position_price_submit(sender) -> None:
    add_position()

def add_position() -> None:
    if textinput_position_value.value == "" and textinput_position_price.value == "":
        log("Bitte Wert oder Preis der Position angeben.")
    else:
        try:
            value = parse_float_value(textinput_position_value.value, position_value_default)
            price = parse_float_value(textinput_position_price.value, position_price_default)
            amount = parse_float_value(textinput_position_amount.value, position_amount_default)
            log(f"Neue Position: {value}, {value}, {value}")
            category = category_provider.get_by_name(select_category.value)
            position = Position(category,
                                select_department.value,
                                value,
                                amount,
                                price,
                                current_bill.page,
                                radiobutton_tax.value)
            bill_data_provider.add_position(current_bill.id, position)
            set_bill_status_to_in_progress()
            textinput_position_amount.value = ""
            textinput_position_value.value = ""
            textinput_position_price.value = ""
        except Exception as e:
            log(f"could not add position: {e}")
            
def parse_float_value(value: str, default: float) -> float:
    if value != "":
        return float(value.replace(",", "."))
    else:
        return default

def set_bill_status_to_in_progress():
    log(f"set status of bill {current_bill.id} to in progress")
    bill_data_provider.set_status(current_bill.id, BillStatus.IN_PROGRESS)
    saved_bill_id = current_bill.id
    update_status_lists()
    current_bill.set_id(saved_bill_id)
    update_positions()
    select_inprogress.value = saved_bill_id
    select_done.value = None
    select_todo.value = None

def delete_position(button: widgets.Button) -> None:
    position_index = select_position.index
    bill_data_provider.delete_position(current_bill.id, position_index)
    if len(bill_data_provider.get_positions(current_bill.id)) == 0:
        bill_data_provider.set_status(current_bill.id, BillStatus.TO_DO)
    update_positions()
    update_status_lists()
    
def move_position_up(button: widgets.Button) -> None:
    log(f"move up bill {current_bill.id} position {select_position.index}")
    bill_data_provider.move_position_up(current_bill.id, select_position.index)
    update_positions()
    
def move_position_down(button: widgets.Button) -> None:
    log(f"move down bill {current_bill.id} position {select_position.index}")
    print(f"move down bill {current_bill.id} position {select_position.index}")
    bill_data_provider.move_position_down(current_bill.id, select_position.index)
    update_positions()
    
def handle_save_comment(button: widgets.Button) -> None:
    log(f"save comment {comment_area.value} for bill {current_bill.id}")
    bill_data_provider.add_comment(current_bill.id, comment_area.value)
    log(f"current bill status: {bill_data_provider.get_status(current_bill.id)}")
    if bill_data_provider.get_status(current_bill.id) == BillStatus.TO_DO:
        set_bill_status_to_in_progress()

def handle_bill_done(button: widgets.Button) -> None:
    log(f"Done Button clicked for {current_bill.id}")
    bill_data_provider.add_comment(current_bill.id, comment_area.value)
    bill_data_provider.set_status(current_bill.id, BillStatus.DONE)
    update_status_lists()

def update_status_lists():
    select_todo.options = bill_data_provider.get_by_status(BillStatus.TO_DO)
    select_inprogress.options = bill_data_provider.get_by_status(BillStatus.IN_PROGRESS)
    select_done.options = bill_data_provider.get_by_status(BillStatus.DONE)

def get_position_description(position: Position) -> str:
    entry_for_unit = f"- x - {position.category.unit}"
    if position.value != 0:
        entry_for_unit = f"{position.amount} x {position.value} {position.category.unit}"
    entry_for_price = "- EUR"
    if position.price != 0:
        entry_for_price = f"{position.price} EUR"
    return f"{position.category.name} {entry_for_unit}, {entry_for_price} (page {position.page_on_pdf+1})"

def update_positions():
    position_descriptions = [get_position_description(p) for p in bill_data_provider.get_positions(current_bill.id)]
    select_position.options = position_descriptions
    
def update_comment_area():
    comment = bill_data_provider.get_comment(current_bill.id)
    comment_area.value = comment

# Layout
left_box = widgets.VBox([select_todo, select_inprogress, select_done])
page_navigation_box = widgets.HBox([button_previous_page, current_page_label, button_next_page])
middle_box = widgets.VBox([image, page_navigation_box])
lower_right_box = widgets.HBox([button_done], layout=Layout())
position_up_down_box = widgets.HBox([button_position_up, button_position_down])
position_button_box = widgets.HBox([button_add_position, button_delete_position])
position_value_input_box = widgets.VBox([widgets.HBox([textinput_position_value, unit_label]), textinput_position_amount, textinput_position_price, radiobutton_tax])

right_box = widgets.VBox([select_position,
                          position_up_down_box,
                          select_department,
                          select_area,
                          select_category,
                          position_value_input_box,
                          position_button_box,
                          lower_right_box,
                          comment_area,
                          button_save_comment,
                          log_output])
main_box = widgets.HBox([left_box, middle_box, right_box])
# Event bindings
select_todo.observe(handle_todo_selection, names="value")
select_inprogress.observe(handle_inprogress_selection, names="value")
select_done.observe(handle_done_selection, names="value")
select_area.observe(handle_area_selection, names="value")
select_category.observe(handle_new_category, names="value")
textinput_position_value.on_submit(on_position_value_submit)
textinput_position_price.on_submit(on_position_price_submit)
textinput_position_amount.on_submit(on_position_amount_submit)

button_position_up.on_click(move_position_up)
button_position_down.on_click(move_position_down)
button_add_position.on_click(handle_add_position_press)
button_delete_position.on_click(delete_position)
button_done.on_click(handle_bill_done)
button_save_comment.on_click(handle_save_comment)
button_previous_page.on_click(handle_previous_page)
button_next_page.on_click(handle_next_page)
# Display main container
display(main_box)
# log("ready to go")
# current_bill.set_id(status_tracker.get_by_status(Status.TO_DO)[0])
if len(bill_data_provider.get_by_status(BillStatus.TO_DO)) > 0:
    first_todo = bill_data_provider.get_by_status(BillStatus.TO_DO)[0]
    current_bill.set_id(first_todo)
    update_positions()
    # select_done.value = None
    # select_inprogress.value = None



/tmp/ipykernel_1948156/4128599353.py:350: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  textinput_position_value.on_submit(on_position_value_submit)
/tmp/ipykernel_1948156/4128599353.py:351: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  textinput_position_price.on_submit(on_position_price_submit)
/tmp/ipykernel_1948156/4128599353.py:352: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  textinput_position_amount.on_submit(on_position_amount_submit)
